# The Easiest Way to Create an Interactive Dashboard in Python

This notebook supports the blog post

**The Easiest Way to Create an Interactive Dashboard in Python. Turn Pandas pipelines into a
dashboard using hvplot `.interactive`**

by *Sophia Yang* and *Marc Skov Madsen*.

![Data App](assets/easy-dataframe-dashboards.gif)

Source: https://github.com/sophiamyang/hvplot_interactive

## Import and configure packages

Please note that in **Colab** you will need to `!pip install panel hvplot`.

In [ ]:
# !pip install panel==0.12.6 hvplot==0.7.3

In [ ]:
import panel as pn

pn.extension('tabulator', sizing_mode="stretch_width")

In [ ]:
import hvplot.pandas
import holoviews as hv

hv.extension('bokeh')

## Define function to determine environment

In [ ]:
def environment():
    try:
        get_ipython()
        return "notebook"
    except:
        return "server"
environment()

## Define Color Palette

In [ ]:
PALETTE = ["#ff6f69", "#ffcc5c", "#88d8b0", ]
pn.Row(
    pn.layout.HSpacer(height=50, background=PALETTE[0]),
    pn.layout.HSpacer(height=50, background=PALETTE[1]),
    pn.layout.HSpacer(height=50, background=PALETTE[2]),
)

## Load Data

In [ ]:
from bokeh.sampledata.autompg import autompg_clean as df
df.head()

## Define DataFrame Pipeline

In [ ]:
(
    df[
        (df.cyl == 4) & 
        (df.mfr.isin(['ford','chevrolet']))
    ]
    .groupby(['origin', 'cyl', 'mfr', 'yr'])['hp'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='yr')
).head(1)

## Make DataFrame Pipeline Interactive

In [ ]:
idf = df.interactive()

Define [Panel widgets](https://panel.holoviz.org/reference/index.html#widgets)

In [ ]:
shared_cylinders_widget = pn.widgets.IntSlider(name='Cylinders (Shared)', start=4, end=8, step=2, value=4)


Combine pipeline and widgets

## Pipe to Table

Check out the [Tabulator Reference Guide](https://panel.holoviz.org/reference/widgets/Tabulator.html) for more inspiration.

## Pipe to hvplot

## Layout using Template

Here we use the [FastListTemplate](https://panel.holoviz.org/reference/templates/FastListTemplate.html#templates-gallery-fastlisttemplate).

In [ ]:
from dashboard_components import InteractiveDashboard

In [ ]:
# --- 3. NUEVO: Crear un panel de solo lectura para mostrar el valor de los cilindros ---
# Esta función se ejecutará cada vez que shared_cylinders_widget.value cambie
@pn.depends(shared_cylinders_widget.param.value)
def display_selected_cylinders(current_cylinders_value):
    return f"**Cilindros Seleccionados Globalmente: {current_cylinders_value}**"

# Crear el panel Markdown que se actualizará reactivamente
selected_cylinders_readout_pane = pn.pane.Markdown(display_selected_cylinders)

# Fila para este panel de solo lectura, para colocarlo en la parte superior del área principal
top_info_row = pn.Row(selected_cylinders_readout_pane, sizing_mode='stretch_width')


unique_mfrs = sorted(list(df['mfr'].unique()))
unique_origins = sorted(list(df['origin'].unique()))

# Configuración para el Dashboard 1 (replicando el filtro mfr anterior)
filter_config1 = [
    {'type': 'multiselect', 'column': 'mfr', 'label': 'Fabricantes', 
     'options': unique_mfrs, 'default_value': unique_mfrs},
    # Puedes añadir más filtros aquí, por ejemplo un filtro de texto para 'name':
    {'type': 'text_filter', 'column': 'origin', 'label': 'Buscar Nombre origen'}
]

y_options_set1 = ['hp', 'weight']

dashboard1 = InteractiveDashboard(
    interactive_dataframe=idf,
    cylinders_widget_from_notebook=shared_cylinders_widget,
    yaxis_options_list=y_options_set1,
    filter_config_list=filter_config1,
    initial_plot_type='line'  # <--- Especifica el tipo de gráfica inicial
).get_view()

In [ ]:
filter_config2 = [
    {'type': 'multiselect', 'column': 'origin', 'label': 'Origen', 
     'options': unique_origins, 'default_value': unique_origins},
    {'type': 'text_filter', 'column': 'name', 'label': 'Buscar Nombre Coche'}
]

y_options_set2 = ['hp', 'weight'] 
dashboard2 = InteractiveDashboard(
    interactive_dataframe=idf,
    cylinders_widget_from_notebook=shared_cylinders_widget,
    yaxis_options_list=y_options_set2,
    filter_config_list=filter_config2,
    initial_plot_type='stacked_line' # <--- Diferente tipo de gráfica inicial
).get_view()

In [ ]:
filter_config3 = [
    {'type': 'multiselect', 'column': 'origin', 'label': 'Origen', 
     'options': unique_origins, 'default_value': unique_origins},
    {'type': 'text_filter', 'column': 'name', 'label': 'Buscar Nombre Coche'}
]
y_options_set3 = ['hp', 'weight'] 
dashboard3 = InteractiveDashboard(
    interactive_dataframe=idf,
    cylinders_widget_from_notebook=shared_cylinders_widget,
    yaxis_options_list=y_options_set3,
    filter_config_list=filter_config3, # Asume que filter_config3 está definido
    initial_plot_type='bar' # <--- Diferente tipo de gráfica inicial
).get_view()

In [ ]:
filter_config4 = [
    {'type': 'multiselect', 'column': 'origin', 'label': 'Origen', 
     'options': unique_origins, 'default_value': unique_origins},
    {'type': 'text_filter', 'column': 'name', 'label': 'Buscar Nombre Coche'}
]

y_axis_options4 = ['hp', 'weight'] 
dashboard4 = InteractiveDashboard(
    interactive_dataframe=idf,
    cylinders_widget_from_notebook=shared_cylinders_widget,
    yaxis_options_list=y_axis_options4,
    filter_config_list=filter_config4, # Asume que filter_config4 está definido
    initial_plot_type='stacked_bar' # <--- Diferente tipo de gráfica inicial
).get_view()

In [ ]:

# --- 1. Template Global para toda la página ---
page_template = pn.template.FastListTemplate(
    title='Multi-Dashboard con Sidebar Global',
    sidebar_width=250, # Ajusta según necesites
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)

page_template.sidebar.append(shared_cylinders_widget)

# --- 4. Organizar los dashboards en una cuadrícula 2x2 ---
# (Puedes ajustar el sizing_mode de las filas y columnas para que se adapten mejor)
grid_of_dashboards = pn.Column(
    pn.Row(dashboard1, sizing_mode='stretch_width'),
    #pn.Row(dashboard3, dashboard4, sizing_mode='stretch_width'),
    sizing_mode='stretch_width' 
)

# --- 5. Añadir la cuadrícula al área 'main' del template global ---
page_template.main.append(top_info_row)
page_template.main.append(grid_of_dashboards)

# --- 6. Hacer servible el Template Global ---
page_template.servable()